In [1]:
# import warnings filter 
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

# Importing Dask packages
import dask.dataframe as dd
import dask.array as da

#import otheer needed python packages
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [ ]:
import os
import wget

# Extracting the dataset online

### To overwrite the previous datasets when re-running
if 'covid19_data.csv' in os.listdir("../data"):
    os.remove('/home/qwerty/Documents/COVID19-time-series-analysis/data/covid19_data.csv')
    path = '/home/qwerty/Documents/COVID19-time-series-analysis/data/covid19_data.csv'
    url =  "https://raw.githubusercontent.com/datasets/covid-19/master/data/time-series-19-covid-combined.csv"
    wget.download(url, path)
else:
    path = '/home/qwerty/Documents/COVID19-time-series-analysis/data/covid19_data.csv'
    url =  "https://raw.githubusercontent.com/datasets/covid-19/master/data/time-series-19-covid-combined.csv"
    wget.download(url, path)
    
# Reading the dataset
data = dd.read_csv('../data/covid19_data.csv',
                   dtype={'Confirmed': 'float64','Deaths': 'float64', 'Lat':'float64', 'long':'float64'})

### Preprocessing the datasets

In [ ]:
data.head()

In [ ]:
data.compute().shape

In [ ]:
data.compute().info()

In [ ]:
#Parsing the 'Date' feature to a date datatype 
data['Date'] = dd.to_datetime(data['Date'], infer_datetime_format=True) 

In [ ]:
data.compute().info()

In [ ]:
# Checking for missing values
data.isnull().sum().compute()

In [ ]:
#Query the features sections with missing values to understand it

# Using the "COnfirmed" feature as case study
data.loc[data['Confirmed'].isnull()].compute()

### This can be done to the other features in the dataset with missing values

In [ ]:
# Query the data with relevant features
dataset = data[['Date','Country/Region',
                   'Confirmed', 'Deaths', 'Recovered']]

In [ ]:
dataset.head()

In [ ]:
datewise_df = dataset.groupby(["Date"]).agg({"Confirmed":'sum',"Recovered":'sum',"Deaths":'sum'})

In [ ]:
datewise_df.head()

In [ ]:
nic = len(dataset["Country/Region"].unique())
cfm_cases = datewise_df["Confirmed"].compute().iloc[-1]
rec = datewise_df["Recovered"].compute().iloc[-1]
deaths = datewise_df["Deaths"].compute().iloc[-1]
active_cases = cfm_cases - rec- deaths
closed_cases = rec + deaths

In [ ]:
def overview():
    print('=== COVID-19 Global Overview ===')
    print(f'== Number of countries which infected:{nic} ==')
    print(f'== Total num of confirmed cases:{cfm_cases} ==')
    print(f'== Total num of recovered cases:{rec} ==')
    print(f'== Total num of deaths:{deaths}')
    print(f'== Total num of Active cases around the world:{active_cases} ==')
    print(f'== Total num of Closed cases around the world:{closed_cases} ==')
    
overview()

In [ ]:
# Plotting the distribution on active cases
plt.figure(figsize=(25,10))
sns.barplot(x=datewise_df.compute().index.date, y = datewise_df["Confirmed"].compute()-
            datewise_df["Recovered"].compute()-datewise_df["Deaths"].compute())
plt.title("Distribution Plot for Active Cases over Date")
plt.xticks(rotation=90)
plt.savefig('../figures/Active Cases Cases over Date.png');

In [ ]:
# Plotting distribution of closed cases
plt.figure(figsize=(26,8))
sns.barplot(x=datewise_df.compute().index.date, y = datewise_df["Recovered"].compute() + 
            datewise_df["Deaths"].compute())
plt.xticks(rotation=90)
plt.title("Distribution Plot for Closed Cases till date over Date")
plt.savefig('../figures/Closed Cases Cases over Date.png');

In [ ]:
woy = datewise_df.compute().index.weekofyear
datewise_df = datewise_df.compute()
datewise_df["WeekOfYear"] = woy

In [ ]:
datewise_df.head()

In [ ]:
week_num=[]
weekwise_confirmed=[]
weekwise_recovered=[]
weekwise_deaths=[]
counter=1
for i in list(datewise_df["WeekOfYear"].unique()):
    weekwise_confirmed.append(datewise_df[datewise_df["WeekOfYear"]==i]["Confirmed"].iloc[-1])
    weekwise_recovered.append(datewise_df[datewise_df["WeekOfYear"]==i]["Recovered"].iloc[-1])
    weekwise_deaths.append(datewise_df[datewise_df["WeekOfYear"]==i]["Deaths"].iloc[-1])
    week_num.append(counter)
    counter=counter+1

In [ ]:
plt.figure(figsize=(18,5))
plt.plot(week_num,weekwise_confirmed,linewidth=3,marker="o", label='Confirmed Cases')
plt.plot(week_num,weekwise_recovered,linewidth=3,marker="^", label='Recovered Cases')
plt.plot(week_num,weekwise_deaths,linewidth=3,marker="+", label='Deaths')
plt.ylabel("Number of Cases")
plt.xlabel("Week Number")
plt.title("Weekly progress of Different Types of Cases")
plt.legend()
plt.savefig('../figures/Weekly progress of Different Types of Cases.png');

In [ ]:
fig, (ax1,ax2) = plt.subplots(1, 2,figsize=(15,5))
sns.barplot(x=week_num,y=pd.Series(weekwise_confirmed).diff().fillna(0),ax=ax1)
sns.barplot(x=week_num,y=pd.Series(weekwise_deaths).diff().fillna(0),ax=ax2)
ax1.set_xlabel("Week Number")
ax2.set_xlabel("Week Number")
ax1.set_ylabel("Number of Confirmed Cases")
ax2.set_ylabel("Number of Death Cases")
ax1.set_title("Weekly increase in Number of Confirmed Cases")
ax2.set_title("Weekly increase in Number of Death Cases")
plt.savefig('../figures/Weekly increase.png');

In [ ]:
plt.figure(figsize=(18,7))
plt.plot(datewise_df["Confirmed"],marker="o",label="Confirmwed cases")
plt.plot(datewise_df["Recovered"],marker="^",label="Recovered cases")
plt.plot(datewise_df["Deaths"],marker="+",label="Death cases")
plt.xticks(rotation="50")
plt.ylabel("Num of patients")
plt.xlabel("Timestamp")
plt.title("Growth rate of confirmed recovered and death cases")
plt.legend()
plt.savefig('../figures/Growth rate.png');

In [ ]:
#Calculating the Mortality Rate and Recovery Rate
datewise_df["Mortality Rate"] = (datewise_df["Deaths"]/datewise_df["Confirmed"])*100 
datewise_df["Recovery Rate"] = (datewise_df["Recovered"]/datewise_df["Confirmed"])*100 
datewise_df["Active Cases"] = (datewise_df["Confirmed"]-datewise_df["Recovered"]-datewise_df["Deaths"]) 
datewise_df["Closed Cases"] = (datewise_df["Deaths"]+datewise_df["Recovered"])

In [ ]:
# ploting mortality and Recovery rate
fig, (ax1,ax2) = plt.subplots(1,2,figsize=(20,6))
ax1.plot(datewise_df["Mortality Rate"],label="Mortality Rate")
ax2.plot(datewise_df["Recovery Rate"],label="Recovery Rate")
ax1.set_title("Moratality Rate")
ax2.set_title("Recovery Rate")
ax1.set_xlabel("Timestamp")
ax2.set_xlabel("Timestamp")
ax1.set_ylabel("Mortality Rate")
ax1.set_xlabel("Recovery Rate")
for tick in ax1.get_xticklabels():
    tick.set_rotation(-70)
for tick in ax2.get_xticklabels():
    tick.set_rotation(-70)
ax1.axhline(datewise_df["Mortality Rate"].mean(),label="Mortality rate mean", linestyle="--", color='black')
ax2.axhline(datewise_df["Recovery Rate"].mean(),label="Recovery rate mean", linestyle="--", color='black')
ax2.legend()
ax1.legend()
plt.savefig('../figures/Mortality and Recovery rate.png');

In [ ]:
plt.figure(figsize=(15,6))
plt.plot(datewise_df["Confirmed"].diff().fillna(0),label="Daily increase in Confiremd Cases",
         linewidth=3,marker="o")
plt.plot(datewise_df["Recovered"].diff().fillna(0),label="Daily increase in recovered Cases",
         linewidth=3,marker="^")
plt.plot(datewise_df["Deaths"].diff().fillna(0),label="Daily increase in deaths Cases",linewidth=3,marker="+")
plt.legend()
plt.xticks(rotation=-70)
plt.ylabel("Daily Increment")
plt.xlabel("Timestamp")
plt.savefig('../figures/Daily increment.png');

In [ ]:
daily_increase_confirm=[]
daily_increase_recovered=[]
daily_increase_deaths=[]
for i in range(datewise_df.shape[0]-1):
    daily_increase_confirm.append(((datewise_df["Confirmed"].iloc[i+1]/datewise_df["Confirmed"].iloc[i])))
    daily_increase_recovered.append(((datewise_df["Recovered"].iloc[i+1]/datewise_df["Recovered"].iloc[i])))
    daily_increase_deaths.append(((datewise_df["Deaths"].iloc[i+1]/datewise_df["Deaths"].iloc[i])))
daily_increase_confirm.insert(0,1)
daily_increase_recovered.insert(0,1)
daily_increase_deaths.insert(0,1)

plt.figure(figsize=(15,6))
plt.plot(datewise_df.index,daily_increase_confirm,label="Growth Factor Confiremd Cases",linewidth=3)
plt.plot(datewise_df.index,daily_increase_recovered,label="Growth Factor Recovered Cases",linewidth=3)
plt.plot(datewise_df.index,daily_increase_deaths,label="Growth Factor Death Cases",linewidth=3)
plt.xlabel("Timestamp")
plt.ylabel("Growth Factor")
plt.title("Growth Factor of different Types of Cases Worldwide")
plt.axhline(1,linestyle='--',color='black',label="Baseline")
plt.xticks(rotation=90)
plt.legend()
plt.savefig('../figures/Growth factor.png');

### Country wise Analysis

In [ ]:
countrywise_df = data[data["Date"]==data["Date"].max()].groupby(["Country/Region"]).agg({
    "Confirmed":"sum","Recovered":"sum","Deaths":"sum", "Lat": "min", "Long": "min"})

countrywise_df["Mortality"]=(countrywise_df["Deaths"]/countrywise_df["Confirmed"])*100
countrywise_df["Recovery"]=(countrywise_df["Recovered"]/countrywise_df["Confirmed"])*100

In [ ]:
countrywise_df.head()

In [ ]:
top_15_confirmed= countrywise_df.Confirmed.nlargest(15).reset_index().compute()
top_15_deaths = countrywise_df.Deaths.nlargest(15).reset_index().compute()
top_15_confirmed = dd.from_pandas(top_15_confirmed, npartitions=1)
top_15_deaths = dd.from_pandas(top_15_deaths, npartitions=1)

In [ ]:
fig, (ax1,ax2) = plt.subplots(2, 1 ,figsize=(10,   12))
sns.barplot(x=top_15_confirmed["Confirmed"].compute(),y=top_15_confirmed["Country/Region"].compute(),ax=ax1)
ax1.set_title("Top 15 countries as per Number of Confirmed Cases")
sns.barplot(x=top_15_deaths["Deaths"].compute(),y=top_15_deaths["Country/Region"].compute(),ax=ax2)
ax2.set_title("Top 15 countries as per Number of Death Cases")
plt.savefig('../figures/Top 15 countries confirmed-Death.png');

In [ ]:
countrywise_df["Active_Cases"] = countrywise_df["Confirmed"] - (countrywise_df["Deaths"] +
                                                                countrywise_df["Recovered"])

In [ ]:
top_15_active = countrywise_df.Active_Cases.nlargest(15).reset_index().compute()
top_15_active = dd.from_pandas(top_15_active, npartitions=1)

In [ ]:
plt.figure(figsize=(12,8))
sns.barplot(x = top_15_active["Active_Cases"].compute(), y = top_15_active["Country/Region"].compute())
plt.title("Top 15 Countries with Most Number of Active Cases")
plt.savefig('../figures/Top 15 Active Cases.png');

In [ ]:
plot_mortality_rate = countrywise_df[countrywise_df["Confirmed"] > 500]
plot_mortality_rate = plot_mortality_rate.Mortality.nlargest(15).reset_index().compute()
plot_mortality_rate = dd.from_pandas(plot_mortality_rate, npartitions=1)
plot_recovery_rate = countrywise_df[countrywise_df["Confirmed"] > 500]
plot_recovery_rate = plot_recovery_rate.Recovered.nlargest(15).reset_index().compute()
plot_recovery_rate = dd.from_pandas(plot_recovery_rate, npartitions=1)

In [ ]:
fig, (ax1,ax2) = plt.subplots(2, 1 , figsize = (11, 12))
sns.barplot(x  = plot_mortality_rate["Mortality"].compute(),y = plot_mortality_rate["Country/Region"].compute(), ax = ax1)
sns.barplot(x  = plot_recovery_rate["Recovered"].compute(),y = plot_recovery_rate["Country/Region"].compute(), ax = ax2)
ax1.set_title("Top 15 Countries according High Mortatlity Rate")
ax1.set_xlabel("Mortality (in Percentage)")
ax2.set_title("Top 15 Countries according High Recovery Rate")
ax2.set_xlabel("Recovery (in Percentage)")
plt.savefig('../figures/Top 15 Mortality and Recovery.png');

In [ ]:
countrywise_df["Survival_Probability"]=(1-(countrywise_df["Deaths"]/countrywise_df["Confirmed"]))*100
top_15_survival=countrywise_df[countrywise_df["Confirmed"]>1000]
top_15_survival=top_15_survival.Survival_Probability.nlargest(15).reset_index().compute()
top_15_survival=dd.from_pandas(top_15_survival, npartitions=1)

In [ ]:
top_15_survival.head()

In [ ]:
bottom_15_survival=countrywise_df[countrywise_df["Confirmed"]>100]
bottom_15_survival=bottom_15_survival.Survival_Probability.nsmallest(15).reset_index().compute()

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1,figsize=(10,15))
sns.barplot(x=top_15_survival["Survival_Probability"].compute(),y=top_15_survival["Country/Region"].compute(),ax=ax1)
ax1.set_title("Top 15 Countries with Maximum Survival Probability having more than 1000 Confiremed Cases")
sns.barplot(x=bottom_15_survival["Survival_Probability"],y=bottom_15_survival["Country/Region"],ax=ax2)
plt.title("Bottom 15 Countries as per Survival Probability")
plt.savefig('../figures/top 15 Survival.png');

In [ ]:
print('Mean Survival Probability across all countries',countrywise_df["Survival_Probability"].mean().compute())
print('Mean Death Probability across all countries',100-countrywise_df["Survival_Probability"].mean().compute())